In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (15.0, 12.0)

import os
import sys
import random
import math
import skimage.io
import matplotlib

# Root directory of the project
ROOT_DIR = os.path.abspath("./")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
# sys.path.append(os.path.join(ROOT_DIR, "coco/"))  # To find local version
import coco

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

Using TensorFlow backend.


In [3]:
dataDir='.'
dataType='train2017'
annFile='%s/annotations/instances_%s.json'%(dataDir,dataType)
# initialize COCO api for instance annotations
cocoDatasetAnn=COCO(annFile)

# get all images containing given categories, select one at random
catIds = cocoDatasetAnn.getCatIds(catNms=['person', 'car'] );
imgIds = cocoDatasetAnn.getImgIds(catIds=catIds );
img = cocoDatasetAnn.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]
print("number of images filterd: ", len(imgIds))
img

loading annotations into memory...
Done (t=10.88s)
creating index...
index created!
number of images filterd:  8519


{'license': 2,
 'file_name': '000000210731.jpg',
 'coco_url': 'http://images.cocodataset.org/train2017/000000210731.jpg',
 'height': 480,
 'width': 640,
 'date_captured': '2013-11-17 06:27:50',
 'flickr_url': 'http://farm3.staticflickr.com/2166/2082460852_cb91dd01b9_z.jpg',
 'id': 210731}

In [4]:
print(catIds)

[1, 3]


In [6]:
config = coco.CocoConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=MODEL_DIR)

In [7]:
# Select weights file to load
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [10]:
#Data set
dataset_train = coco.CocoDataset()
dataset_train.load_coco(dataset_dir=IMAGE_DIR, subset="train", year="2017", class_ids=[1,3])

loading annotations into memory...
Done (t=14.95s)
creating index...
index created!


In [12]:
import imgaug
dataset_train.prepare()
augmentation = imgaug.augmenters.Fliplr(0.5)

In [16]:
# Validation dataset
dataset_val = coco.CocoDataset()
val_type = "val"
dataset_val.load_coco(IMAGE_DIR, val_type, year="2017", class_ids=[1,3])
dataset_val.prepare()

loading annotations into memory...
Done (t=0.27s)
creating index...
index created!


In [17]:
model.train(dataset_train, dataset_val,
                    learning_rate=config.LEARNING_RATE,
                    epochs=120,
                    layers='5+',
                    augmentation=augmentation)


Starting at epoch 0. LR=0.001

Checkpoint Path: /home/aazmi/Documents/MSC INF/Spring2019/cs231nProject/logs/coco20190630T1707/mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
res5a_branch2a         (Conv2D)
bn5a_branch2a          (BatchNorm)
res5a_branch2b         (Conv2D)
bn5a_branch2b          (BatchNorm)
res5a_branch2c         (Conv2D)
res5a_branch1          (Conv2D)
bn5a_branch2c          (BatchNorm)
bn5a_branch1           (BatchNorm)
res5b_branch2a         (Conv2D)
bn5b_branch2a          (BatchNorm)
res5b_branch2b         (Conv2D)
bn5b_branch2b          (BatchNorm)
res5b_branch2c         (Conv2D)
bn5b_branch2c          (BatchNorm)
res5c_branch2a         (Conv2D)
bn5c_branch2a          (BatchNorm)
res5c_branch2b         (Conv2D)
bn5c_branch2b          (BatchNorm)
res5c_branch2c         (Conv2D)
bn5c_branch2c          (BatchNorm)
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5             

/home/aazmi/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/aazmi/anaconda3/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/120


ERROR:root:Error processing image {'id': 235543, 'source': 'coco', 'path': '/home/aazmi/Documents/MSC INF/Spring2019/cs231nProject/images/train2017/000000235543.jpg', 'width': 433, 'height': 640, 'annotations': [{'segmentation': [[44.68, 281.8, 31.71, 248.65, 44.68, 222.7, 61.98, 206.85, 87.93, 201.08, 116.76, 208.29, 126.85, 241.44, 126.85, 281.8, 119.64, 293.33, 147.03, 296.22, 155.68, 297.66, 164.32, 303.42, 172.97, 312.07, 187.39, 319.28, 158.56, 343.78, 160.0, 379.82, 201.8, 407.21, 221.98, 427.39, 200.36, 447.57, 66.31, 440.36, 33.15, 425.95, 24.5, 418.74, 11.53, 407.21, 10.09, 359.64, 23.06, 330.81]], 'area': 30789.0821, 'iscrowd': 0, 'image_id': 235543, 'bbox': [10.09, 201.08, 211.89, 246.49], 'category_id': 1, 'id': 423164}, {'segmentation': [[354.3, 278.17, 363.24, 262.74, 381.92, 232.68, 424.17, 215.62, 429.85, 251.36, 424.98, 253.8, 420.1, 264.36, 420.1, 271.67, 409.54, 278.17, 392.48, 275.73, 395.73, 288.73, 390.86, 291.17, 380.3, 283.86, 378.67, 273.3, 372.99, 266.8, 356.

FileNotFoundError: [Errno 2] No such file or directory: '/home/aazmi/Documents/MSC INF/Spring2019/cs231nProject/images/train2017/000000235543.jpg'ERROR:root:Error processing image {'id': 235543, 'source': 'coco', 'path': '/home/aazmi/Documents/MSC INF/Spring2019/cs231nProject/images/train2017/000000235543.jpg', 'width': 433, 'height': 640, 'annotations': [{'segmentation': [[44.68, 281.8, 31.71, 248.65, 44.68, 222.7, 61.98, 206.85, 87.93, 201.08, 116.76, 208.29, 126.85, 241.44, 126.85, 281.8, 119.64, 293.33, 147.03, 296.22, 155.68, 297.66, 164.32, 303.42, 172.97, 312.07, 187.39, 319.28, 158.56, 343.78, 160.0, 379.82, 201.8, 407.21, 221.98, 427.39, 200.36, 447.57, 66.31, 440.36, 33.15, 425.95, 24.5, 418.74, 11.53, 407.21, 10.09, 359.64, 23.06, 330.81]], 'area': 30789.0821, 'iscrowd': 0, 'image_id': 235543, 'bbox': [10.09, 201.08, 211.89, 246.49], 'category_id': 1, 'id': 423164}, {'segmentation': [[354.3, 278.17, 363.24, 262.74, 381.92, 232.68, 424.17, 215.62, 429.85, 251.36, 424.98, 253.

...

FileNotFoundError: [Errno 2] No such file or directory: '/home/aazmi/Documents/MSC INF/Spring2019/cs231nProject/images/train2017/000000501515.jpg'
ERROR:root:Error processing image {'id': 501515, 'source': 'coco', 'path': '/home/aazmi/Documents/MSC INF/Spring2019/cs231nProject/images/train2017/000000501515.jpg', 'width': 612, 'height': 612, 'annotations': [{'segmentation': [[162.28, 186.63, 174.66, 240.26, 174.66, 258.14, 154.03, 263.64, 114.15, 266.39, 52.26, 274.64, 2.75, 288.4, 1.38, 412.17, 67.39, 413.55, 66.01, 518.07, 63.26, 603.34, 339.69, 603.34, 339.69, 500.19, 386.45, 573.08, 409.83, 599.21, 492.35, 599.21, 431.84, 515.32, 409.83, 467.18, 393.33, 427.3, 378.2, 388.79, 358.95, 320.03, 352.07, 306.28, 328.69, 291.15, 305.31, 287.02, 265.43, 267.77, 269.56, 215.51, 294.31, 185.25, 284.68, 148.12, 280.56, 113.74, 258.55, 83.48, 242.05, 71.1, 220.04, 68.35, 187.04, 73.85, 167.78, 104.11, 167.78, 116.49, 170.53, 149.49, 171.91, 171.5, 144.4, 181.12], [145.78, 163.25, 132.03, 166.0,

FileNotFoundError: [Errno 2] No such file or directory: '/home/aazmi/Documents/MSC INF/Spring2019/cs231nProject/images/train2017/000000373089.jpg'

ERROR:root:Error processing image {'id': 373089, 'source': 'coco', 'path': '/home/aazmi/Documents/MSC INF/Spring2019/cs231nProject/images/train2017/000000373089.jpg', 'width': 446, 'height': 640, 'annotations': [{'segmentation': [[444.9, 214.15, 436.46, 214.39, 430.39, 218.07, 428.73, 224.02, 430.99, 227.11, 435.51, 227.11, 437.77, 229.96, 443.11, 228.77, 443.35, 227.94, 446.0, 228.18]], 'area': 205.60540000000015, 'iscrowd': 0, 'image_id': 373089, 'bbox': [428.73, 214.15, 17.27, 15.81], 'category_id': 3, 'id': 1345613}, {'segmentation': [[427.61, 224.38, 423.23, 222.07, 422.54, 218.84, 422.08, 213.53, 427.15, 208.92, 445.61, 208.46, 446.0, 213.76, 443.07, 214.23, 435.92, 214.23, 430.61, 218.15]], 'area': 194.1372500000001, 'iscrowd': 0, 'image_id': 373089, 'bbox': [422.08, 208.46, 23.92, 15.92], 'category_id': 3, 'id': 1346271}, {'segmenta

FileNotFoundError: [Errno 2] No such file or directory: '/home/aazmi/Documents/MSC INF/Spring2019/cs231nProject/images/train2017/000000373089.jpg'
ERROR:root:Error processing image {'id': 373089, 'source': 'coco', 'path': '/home/aazmi/Documents/MSC INF/Spring2019/cs231nProject/images/train2017/000000373089.jpg', 'width': 446, 'height': 640, 'annotations': [{'segmentation': [[444.9, 214.15, 436.46, 214.39, 430.39, 218.07, 428.73, 224.02, 430.99, 227.11, 435.51, 227.11, 437.77, 229.96, 443.11, 228.77, 443.35, 227.94, 446.0, 228.18]], 'area': 205.60540000000015, 'iscrowd': 0, 'image_id': 373089, 'bbox': [428.73, 214.15, 17.27, 15.81], 'category_id': 3, 'id': 1345613}, {'segmentation': [[427.61, 224.38, 423.23, 222.07, 422.54, 218.84, 422.08, 213.53, 427.15, 208.92, 445.61, 208.46, 446.0, 213.76, 443.07, 214.23, 435.92, 214.23, 430.61, 218.15]], 'area': 194.1372500000001, 'iscrowd': 0, 'image_id': 373089, 'bbox': [422.08, 208.46, 23.92, 15.92], 'category_id': 3, 'id': 1346271}, {'segmentat

FileNotFoundError: [Errno 2] No such file or directory: '/home/aazmi/Documents/MSC INF/Spring2019/cs231nProject/images/train2017/000000501515.jpg'



ERROR:root:Error processing image {'id': 373089, 'source': 'coco', 'path': '/home/aazmi/Documents/MSC INF/Spring2019/cs231nProject/images/train2017/000000373089.jpg', 'width': 446, 'height': 640, 'annotations': [{'segmentation': [[444.9, 214.15, 436.46, 214.39, 430.39, 218.07, 428.73, 224.02, 430.99, 227.11, 435.51, 227.11, 437.77, 229.96, 443.11, 228.77, 443.35, 227.94, 446.0, 228.18]], 'area': 205.60540000000015, 'iscrowd': 0, 'image_id': 373089, 'bbox': [428.73, 214.15, 17.27, 15.81], 'category_id': 3, 'id': 1345613}, {'segmentation': [[427.61, 224.38, 423.23, 222.07, 422.54, 218.84, 422.08, 213.53, 427.15, 208.92, 445.61, 208.46, 446.0, 213.76, 443.07, 214.23, 435.92, 214.23, 430.61, 218.15]], 'area': 194.1372500000001, 'iscrowd': 0, 'image_id': 373089, 'bbox': [422.08, 208.46, 23.92, 15.92], 'category_id': 3, 'id': 1346271}, {'segmen

FileNotFoundError: [Errno 2] No such file or directory: '/home/aazmi/Documents/MSC INF/Spring2019/cs231nProject/images/train2017/000000373089.jpg'